In [2]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import os
%matplotlib inline

In [3]:
path_to_data = '../xview_data_small'

### Get all bounding boxes

In [10]:
# Get all coords, chips, and classes
coords, chips, classes = wv.get_labels('{}xView_train.geojson'.format(path_to_data))

100%|██████████| 601937/601937 [00:02<00:00, 203088.46it/s]


In [11]:
# Sanity check stats
print('Shapes')
print('coords: {}'.format(coords.shape))
print('chips: {}'.format(chips.shape))
print('classes: {}'.format(classes.shape))

print('\nAn example')
print('coords[1]: {}'.format(coords[1]))
print('chips[1]: {}'.format(chips[1]))
print('classes[1]: {}'.format(classes[1]))

print('\nNum unique chips: {}'.format(len(set(chips))))
print('Num unique classes: {}'.format(len(set(classes))))

Shapes
coords: (601937, 4)
chips: (601937,)
classes: (601937,)

An example
coords[1]: [2720. 2233. 2760. 2288.]
chips[1]: 2355.tif
classes[1]: 73.0

Num unique chips: 847
Num unique classes: 62


### Create names (classes) file
Step 3 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Remap class numbers to be consecutive and to start from 0, then write all class labels to `*.names` file.

In [22]:
old_to_new_labels = {}
old_labels = {}
new_labels = {}
new_num = 0

# Remap all explicitly named classes
with open('xview_class_labels.txt') as f:
    for row in csv.reader(f):
        arr = row[0].split(":")
        old_num = int(arr[0])
        label_str = arr[1]
        
        old_to_new_labels[old_num] = new_num 
        old_labels[old_num] = label_str
        new_labels[new_num] = label_str
        
        new_num += 1

# Remap unnamed classes
for c in set(classes):
    if c not in old_labels:
        old_to_new_labels[int(c)] = new_num
        new_labels[new_num] = "Unknown label {}".format(new_num)
        new_num += 1

In [23]:
print(old_to_new_labels)
print(labels)

{11: 0, 12: 1, 13: 2, 15: 3, 17: 4, 18: 5, 19: 6, 20: 7, 21: 8, 23: 9, 24: 10, 25: 11, 26: 12, 27: 13, 28: 14, 29: 15, 32: 16, 33: 17, 34: 18, 35: 19, 36: 20, 37: 21, 38: 22, 40: 23, 41: 24, 42: 25, 44: 26, 45: 27, 47: 28, 49: 29, 50: 30, 51: 31, 52: 32, 53: 33, 54: 34, 55: 35, 56: 36, 57: 37, 59: 38, 60: 39, 61: 40, 62: 41, 63: 42, 64: 43, 65: 44, 66: 45, 71: 46, 72: 47, 73: 48, 74: 49, 76: 50, 77: 51, 79: 52, 83: 53, 84: 54, 86: 55, 89: 56, 91: 57, 93: 58, 94: 59, 75: 60, 82: 61}
{0: 'Fixed-wing Aircraft', 1: 'Small Aircraft', 2: 'Cargo Plane', 3: 'Helicopter', 4: 'Passenger Vehicle', 5: 'Small Car', 6: 'Bus', 7: 'Pickup Truck', 8: 'Utility Truck', 9: 'Truck', 10: 'Cargo Truck', 11: 'Truck w/Box', 12: 'Truck Tractor', 13: 'Trailer', 14: 'Truck w/Flatbed', 15: 'Truck w/Liquid', 16: 'Crane Truck', 17: 'Railway Vehicle', 18: 'Passenger Car', 19: 'Cargo Car', 20: 'Flat Car', 21: 'Tank car', 22: 'Locomotive', 23: 'Maritime Vessel', 24: 'Motorboat', 25: 'Sailboat', 26: 'Tugboat', 27: 'Barg

In [32]:
# Create new *.names file listing all of the names for the classes in our dataset
label_file = open("{}/yolo_data/xview.names".format(path_to_data), "a")
for i in new_labels:
    label_file.write('{}\n'.format(new_labels[i]))
label_file.close()

In [24]:
# Remap classes array
remapped_classes = list(map(lambda c: old_to_new_labels[int(c)], classes))
remapped_classes = np.array(remapped_classes)

In [25]:
print(remapped_classes.shape)

(601937,)


### Chip each image
Step 1 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Output all chip tifs and associated label txt files. Each image's label file must be locatable by simply replacing `/images/*.jpg` with `/labels/*.txt` in its pathname. Additional requirements:
* One file per image (if no objects in image, no label file is required).
* One row per object.
* Each row is `class x_center y_center width height` format.
* Box coordinates must be in normalized xywh format (from 0 - 1).
* Classes are zero-indexed (start from 0).

In [66]:
images = ['53.tif']
chip_size = (500, 500)
path_to_save = '{}/yolo_data'.format(path_to_data)

# For each image in directory
for img_name in os.listdir('{}/train_images'.format(path_to_data)):
    if not img_name.endswith(".tif"):
        continue
    
    # Get full image
    print('Processing {}'.format(img_name))
    full_path = '{}/train_images/{}'.format(path_to_data, img_name)
    arr = wv.get_image(full_path)
    
    # Get chip arrays, bounding boxes, and classes
    c_img, c_box, c_cls = wv.chip_image(img=arr, coords=coords[chips==img_name], classes=classes[chips==img_name], shape=chip_size)
    num_chips = c_img.shape[0]
    print("\tNum chips: {}".format(num_chips))
        
    for i in range(num_chips):
        filename = '{}_{}'.format(img_name.split('.')[0], i)
        
        # Write chip JPG
        chip_arr = c_img[i]
        chip_img = Image.fromarray(chip_arr)
        chip_img.save("{}/images/{}.jpeg".format(path_to_save, filename))
        
        # Write chip label txt
        chip_bboxes = c_box[i]
        chip_classes = c_cls[i]
        num_bboxes = len(chip_classes)
#         print('\t\tNum bounding boxes: {}'.format(num_bboxes))
        
        labels_str = ""
        for j in range(num_bboxes):
            # bounding box format: xmin, ymin, xmax, ymax
            xmin, ymin, xmax, ymax = chip_bboxes[j]
            bbox_class = int(chip_classes[j])
            
            # row format: class x_center y_center width height
            if bbox_class == 0:
                labels_str += "0 0 0 0 0\n"
            else:
                labels_str += '{} {} {} {} {}\n'.format(old_to_new_labels[bbox_class],
                                                       (xmax + xmin)/2 / chip_size[1], 
                                                       (ymax + ymin)/2 / chip_size[0],
                                                       (xmax - xmin) / chip_size[1],
                                                       (ymax - ymin) / chip_size[0])
        
        label_file = open("{}/labels/{}.txt".format(path_to_save, filename), "a")
        label_file.write(labels_str)
        label_file.close()
        
        chip_num += 1

Processing 905.tif
	Num chips: 36
Processing 99.tif
	Num chips: 30
Processing 819.tif
	Num chips: 36
Processing 893.tif
	Num chips: 42
Processing 847.tif
	Num chips: 36
Processing 888.tif
	Num chips: 36
Processing 903.tif
	Num chips: 36
Processing 53.tif
	Num chips: 30
Processing 8.tif
	Num chips: 36
Processing 880.tif
	Num chips: 36


### Create test and train files
Step 2 of "Train Custom Data" [wiki](https://github.com/ultralytics/yolov3/wiki/Example:-Transfer-Learning). Create train and test `*.txt` files. Each row contains a path to an image, and remember one label must also exist in a corresponding /labels folder for each image that has targets.

In [8]:
path_to_save = '{}/yolo_data'.format(path_to_data)

# As a test, use the same random subset of data for train and test
num_images = 20

file = open("{}/xview_small_imgs.txt".format(path_to_save), "a")

all_imgs = os.listdir('{}/images'.format(path_to_save))
for img_filename in np.random.choice(all_imgs, num_images):
    file.write('../xview_data_small/yolo_data/images/{}\n'.format(img_filename))

file.close()